This gets a unix timestamp input, converts it to a pandas compatible datetime, runs it against known hourly data, and returns a temperature. Linear interpolation will be used between two hourly data points. 

Maybe some optimization could be done if the two hourly temps have changed less than a certain amount.

In [2]:
from netCDF4 import Dataset, MFDataset, num2date
import pandas as pd
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib as ply
from datetime import datetime, timedelta

at_data = Dataset("../data/air_temperature.nc")
air_temperature = at_data.variables['air_temperature'][:]
time = at_data.variables['time'][:]

In [5]:
time_var = at_data.variables['time']
dates = num2date(time[:], time_var.units)
dates_pd = pd.to_datetime(dates)

In [6]:
df = pd.DataFrame(dates_pd,columns=['Date'])
df['air_temperature'] = air_temperature[:]
df = df.set_index('Date')

In [86]:
pumaRange = df.loc['2018-10':'2019-05']
type(pumaRange.index)

pandas.core.indexes.datetimes.DatetimeIndex

The way the algorithm will work is that it will check the timestamp that is input and see if it is +/- 15 minutes from the top of the hour. If it is, then the returned temperature will just be that hourly datapoint. If it is not, then two weather points before and after the input time, then the returned temperature will be an interpolation between the two

In [150]:
def outdoorTemp(inUnix):
    pumaPoint = datetime.utcfromtimestamp(inUnix)
    nearestIndex = pumaRange.index.get_loc(pumaPoint,method='nearest')
    
    print(nearestIndex)
    nearestTemp = pumaRange.iloc[nearestIndex]
    print(str(nearestTemp.name) + ': ' + str(nearestTemp.air_temperature) + 'F')

    timeDiff = nearestTemp.name - pumaPoint #timeDelta
    #in this expression, when the point is ahead, the day=0
    #when the point is behind, the day=-1
    print(timeDiff)
    
    if abs(timeDiff) < timedelta(minutes = 15):
        #less than 15 minutes from an hour
        return(nearestTemp.air_temperature)
    elif timeDiff.days == -1:
        #greater than 15 minutes from the hour
        #and goes to previous hour
        return('¯\_(ツ)_/¯')
    else:
        #greater than 15 minutes from the hour
        #and goes to next hour
        return('¯\_(ツ)_/¯')
    
print(outdoorTemp(1546386804)) #2019-1-1 23:53:24 goes to the next hour, less that 15 min
print('\r\n')
print(outdoorTemp(1546381404)) #2019-1-1 22:23:24 goes to the previous hour, more than 15
print('\r\n')
print(outdoorTemp(1546380612)) #2019-1-1 22:10:12 goes to the previous hour, less than 15
print('\r\n')
print(outdoorTemp(1546382112)) #2019-1-1 22:35:12 goes to the next hour, more than 15

2232
2019-01-02 00:00:00: 30.02F
0 days 00:06:36
30.02


2230
2019-01-01 22:00:00: 25.88F
-1 days +23:36:36
¯\_(ツ)_/¯


2230
2019-01-01 22:00:00: 25.88F
-1 days +23:49:48
25.88


2231
2019-01-01 23:00:00: 26.24F
0 days 00:24:48
¯\_(ツ)_/¯
